# custom exceptions for
- numpy
- pandas
- requests
- sqlite3

### Custom Exception


In [71]:

class DBMgrError(Exception):
    pass

raise DBMgrError("Database operation failed.")


DBMgrError: Database operation failed.

### Custom Exception - numpy


```text
BaseException
 └── Exception
      ├── LookupError
      │    └── IndexError
      │         └── numpy.AxisError
      ├── ValueError
      │    └── numpy.AxisError
      ├── TypeError
      │    └── numpy.UFuncTypeError
      ├── RuntimeError
      │    └── numpy.core._exceptions.TooHardError
      └── MemoryError
           └── numpy.core._exceptions._ArrayMemoryError

```

In [1]:
import numpy as np

In [2]:
arr = np.array([1, 2, 3])
arr.sum(axis=2)  # Invalid axis

AxisError: axis 2 is out of bounds for array of dimension 1

In [3]:
import numpy as np

arr = np.array([1, 2, 3])
result = np.add(arr, "string")  # Invalid operation: adding a string to an array


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('int64'), dtype('<U6')) -> None

### Custom Exception - Pandas


```text
BaseException
 └── Exception
      ├── ValueError
      │    ├── pandas.errors.EmptyDataError
      │    ├── pandas.errors.ParserError
      │    └── pandas.errors.MergeError
      ├── TypeError
      │    └── pandas.errors.UnsupportedFunctionCall
```

In [4]:
import pandas.errors
import pandas as pd

In [5]:
from sqlite3 import connect
conn = connect(':memory:')
with conn:
    pd.read_sql('select * test', conn)

DatabaseError: Execution failed on sql 'select * test': near "test": syntax error

In [14]:
from sqlite3 import connect
conn = connect(':memory:')
with conn:
    pd.DataFrame({"item":[1,2]}).to_sql(123, conn, if_exists='append', index=False)

TypeError: object of type 'int' has no len()

In [96]:
import pandas as pd

# Create a simple DataFrame
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6]
})

try:
    # Attempt to access a non-existent column 'C'
    print(df['C'])
except LookupError as e:
    print(f"{type(e) = }, {e}")

type(e) = <class 'KeyError'>, 'C'


### Custom Exception - requests

```text
OSError
 └── requests.exceptions.RequestException
      ├── requests.exceptions.HTTPError
      ├── requests.exceptions.ConnectionError
      │    └── requests.exceptions.ProxyError
      ├── requests.exceptions.Timeout
      ├── requests.exceptions.TooManyRedirects
      ├── requests.exceptions.URLRequired
      ├── requests.exceptions.InvalidURL
      │    ├── requests.exceptions.MissingSchema
      │    └── requests.exceptions.InvalidSchema
      ├── requests.exceptions.ChunkedEncodingError
      ├── requests.exceptions.ContentDecodingError
      └── requests.exceptions.SSLError
```

In [1]:
import requests
from requests.exceptions import RequestException

In [110]:
# HTTPError: Raised for unsuccessful HTTP status codes (e.g., 404)
try:
    response = requests.get("https://httpbin.org/status/404")
    response.raise_for_status()
except RequestException as e:
    print(f"{type(e) = }, {e}")



type(e) = <class 'requests.exceptions.HTTPError'>, 404 Client Error: NOT FOUND for url: https://httpbin.org/status/404


In [107]:
# ConnectionError: Raised when a connection to the server fails
try:
    response = requests.get("http://nonexistent.url")
except OSError as e:
    print(f"{type(e) = }, {e}")



type(e) = <class 'requests.exceptions.ConnectionError'>, HTTPConnectionPool(host='nonexistent.url', port=80): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000002A75F700950>: Failed to resolve 'nonexistent.url' ([Errno 11001] getaddrinfo failed)"))


In [108]:
# Timeout: Raised when a request times out
try:
    response = requests.get("https://httpbin.org/delay/10", timeout=2)
except OSError as e:
    print(f"{type(e) = }, {e}")



type(e) = <class 'requests.exceptions.ReadTimeout'>, HTTPSConnectionPool(host='httpbin.org', port=443): Read timed out. (read timeout=2)


In [112]:
# TooManyRedirects: Raised when the maximum number of redirects is exceeded
try:
    response = requests.get("https://httpbin.org/redirect/100", allow_redirects=True)
except OSError as e:
    print(f"{type(e) = }, {e}")


type(e) = <class 'requests.exceptions.TooManyRedirects'>, Exceeded 30 redirects.


In [120]:
# URLRequired: Raised when a valid URL is not provided
try:
    response = requests.get("")
except OSError as e:
    print(f"{type(e) = }, {e}")


# InvalidURL: Raised when the URL is invalid
try:
    response = requests.get("invalid-url")
except OSError as e:
    print(f"{type(e) = }, {e}")


# MissingSchema: Raised when the URL is missing a schema (e.g., http://)
try:
    response = requests.get("example.com")  # Missing "http://"
except OSError as e:
    print(f"{type(e) = }, {e}")


type(e) = <class 'requests.exceptions.MissingSchema'>, Invalid URL '': No scheme supplied. Perhaps you meant https://?
type(e) = <class 'requests.exceptions.MissingSchema'>, Invalid URL 'invalid-url': No scheme supplied. Perhaps you meant https://invalid-url?
type(e) = <class 'requests.exceptions.MissingSchema'>, Invalid URL 'example.com': No scheme supplied. Perhaps you meant https://example.com?


In [121]:

# InvalidSchema: Raised when the URL has an unsupported schema
try:
    response = requests.get("ftp://example.com")  # Unsupported schema
except OSError as e:
    print(f"{type(e) = }, {e}")


type(e) = <class 'requests.exceptions.InvalidSchema'>, No connection adapters were found for 'ftp://example.com'


In [122]:

# SSLError: Raised when there is an SSL certificate error
try:
    response = requests.get("https://expired.badssl.com/")
except OSError as e:
    print(f"{type(e) = }, {e}")

type(e) = <class 'requests.exceptions.SSLError'>, HTTPSConnectionPool(host='expired.badssl.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)')))


In [8]:
try:
    response = requests.get("https://httpbirastst")
    response.raise_for_status()
except requests.exceptions.HTTPError as e:
    if e.response.status_code == 429:
        print("Rate limit exceeded")
        raise e
    elif e.response.status_code == 500:
        print("Internal server error")
        raise e
    else:
        print("HTTP error occurred:")
        raise e
except requests.exceptions.RequestException as e:
    print("Request error occurred:")
    raise e

Request error occurred:


ConnectionError: HTTPSConnectionPool(host='httpbirastst', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000028F493D0B90>: Failed to resolve 'httpbirastst' ([Errno 11001] getaddrinfo failed)"))

### Custom Exception - sqlite3

```text
Exception
 └── sqlite3.Error
      ├── sqlite3.Warning
      ├── sqlite3.DatabaseError
      │    ├── sqlite3.IntegrityError
      │    ├── sqlite3.ProgrammingError
      │    ├── sqlite3.OperationalError
      │    ├── sqlite3.NotSupportedError
      │    └── sqlite3.DataError
      ├── sqlite3.InterfaceError
      └── sqlite3.InternalError
```

In [124]:
import sqlite3

In [125]:
# IntegrityError: Raised when a database integrity constraint is violated
try:
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE test (id INTEGER PRIMARY KEY, value TEXT UNIQUE)")
    cursor.execute("INSERT INTO test (id, value) VALUES (1, 'A')")
    cursor.execute("INSERT INTO test (id, value) VALUES (2, 'A')")  # Violates UNIQUE constraint
except sqlite3.DatabaseError as e:
    print(f"{type(e) = }, {e}")


type(e) = <class 'sqlite3.IntegrityError'>, UNIQUE constraint failed: test.value


In [138]:

# DataError: Raised for errors related to invalid data
try:
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE test (id INTEGER, value INTEGER) STRICT")
    cursor.execute("INSERT INTO test (id, value) VALUES (1.2, 'invalid_integer3')")  # Type mismatch
except sqlite3.DatabaseError as e:
    print(f"{type(e) = }, {e}")
finally:
    conn.close()

type(e) = <class 'sqlite3.IntegrityError'>, cannot store REAL value in INTEGER column test.id


In [126]:

# ProgrammingError: Raised for programming errors, such as invalid SQL syntax
try:
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()
    cursor.execute("INVALID SQL SYNTAX")  # Invalid SQL
except sqlite3.DatabaseError as e:
    print(f"{type(e) = }, {e}")


type(e) = <class 'sqlite3.OperationalError'>, near "INVALID": syntax error


In [127]:

# OperationalError: Raised for operational errors, such as missing tables
try:
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM non_existent_table")  # Table does not exist
except sqlite3.DatabaseError as e:
    print(f"{type(e) = }, {e}")


type(e) = <class 'sqlite3.OperationalError'>, no such table: non_existent_table


In [141]:
import sqlite3

try:
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()
    
    # Passing incorrect number of parameters to the query
    cursor.execute("CREATE TABLE test (id INTEGER, value TEXT)")
    cursor.execute("INSERT INTO test (id, value) VALUES (?, ?)", (1,))  # Missing one parameter
except sqlite3.DatabaseError as e:
    print(f"{type(e) = }, {e}")

type(e) = <class 'sqlite3.ProgrammingError'>, Incorrect number of bindings supplied. The current statement uses 2, and there are 1 supplied.


In [150]:
try:
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()
    conn.close()  # Close the connection
    
    # Attempting to execute a query after the connection is closed
    cursor.execute("CREATE TABLE test (id INTEGER, value TEXT)")
except sqlite3.DatabaseError as e:
    print(f"{type(e) = }, {e}")

type(e) = <class 'sqlite3.ProgrammingError'>, Cannot operate on a closed database.


In [145]:
try:
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()
    cursor.execute(12345)  # SQL query must be a string, not an integer
except TypeError as e:
    print(f"{type(e) = }, {e}")

type(e) = <class 'TypeError'>, execute() argument 1 must be str, not int
